In [ ]:
"""
把从上交所和深交所下载的年报pdf，提取文本

为了加速，采用多进程并发方式
"""

In [ ]:
#!jupyter nbconvert --to script pdf_to_txt.ipynb
import os
def pdf_to_txt(pdf_file,txt_dir='./'):
    import subprocess
    import os
    try:
        txt_filename=txt_dir+os.path.basename(pdf_file).replace('pdf','txt')
#         out_bytes = subprocess.check_output(['which','pdf2txt.py'])
#         pdf2txt=out_bytes.decode("utf-8").replace('\n','')
#         if not pdf2txt:
#             print('pdf2txt.py 不存在，程序退出<c')
#             quit()
#         subprocess.run(["python", pdf2txt, "-o",txt_filename ,pdf_file])
        subprocess.run(["pdf2txt.py", "-o",txt_filename ,pdf_file])
    except subprocess.CalledProcessError as e:
        out_bytes = e.output       # Output generated before error
        code      = e.returncode   # Return code
        print(out_bytes,code)
    except:
        out_bytes = e.output       # Output generated before error
        code      = e.returncode   # Return code
        print("***=",out_bytes,code)
     

    

# pdf_dir ='../scraping/pdf/'
# pdf_file=pdf_dir+'000780_2006.pdf' 
# pdf_to_txt(pdf_file)

In [ ]:
# find files

def find_files(pdf_dir):
    return [pdf_dir+f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

pdf_files = find_files(pdf_dir)
print(len(pdf_files))
print(pdf_files[:4])

In [ ]:


base_dir='../scraping/content/'
import time  
from multiprocessing import Process, Queue
from tqdm import tqdm

q = Queue()

def f(q: Queue):
    while True:
        pdf_file = q.get()
        t0 = time.time()
        print(f'进程{os.getpid()} 开始转换文件：{pdf_file}')
        pdf_to_txt(pdf_file,base_dir)
        print(f'进程{os.getpid()} 转换文件成功，用时(秒):{time.time()-t0}')


            
def loop_task():
    import time  
    for pdf_file in tqdm(pdf_files):
        if q.qsize()> 10:
            print("sleep for queue is not empty")
            time.sleep(3)  # sleep 10秒
        else:
            q.put(pdf_file)


# 启动四个进程进行处理
for i in range(6):
    p = Process(target=f, args=(q,))
    p.start()

loop_task()

    